<a href="https://colab.research.google.com/github/Masfiqur-Mim/Auto_Color/blob/master/Changed_Transfer_learning_VGG16_IDG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


CopyData.py

In [ ]:
!cp -r '/content/drive/My Drive/Dataset' '/content/'

#Import libraries

In [ ]:
import keras.backend as K
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, ReduceLROnPlateau
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile
#print(tf.__version__)

In [ ]:
batch_size = 64
IMG_SIZE = 64

Datagen.py

In [ ]:
import numpy as np
import cv2

import keras
 
class ImageDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_files, dim, batch_size=64, shuffle=True):
        'Initialization'
        
        self.list_files = list_files
        self.dim = dim
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_files) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        list_files_batch = [self.list_files[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_files_batch)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_files))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def normalize_batch(self, imgs):
        imgs = np.array(imgs, dtype=np.float32)
        imgs /= 255
        return imgs
                                                            
    def denormalize_batch(self, imgs, should_clip=True):
        #imgs= imgs * self.std + self.mean 
        if should_clip:
            imgs = np.clip(imgs,0,1)
        imgs *= 255
        return imgs

    def __data_generation(self, list_files_batch):
        'Generates data containing batch_size samples' 
        X = []
        Y = []
        for i, fname in enumerate(list_files_batch):
            img = cv2.imread(fname)
            img = cv2.resize(img, (self.dim[1], self.dim[0]))
            # print(img.shape)
            X.append(img)
        X = self.normalize_batch(X)
        lab_batch = rgb2lab(X)
        X_batch = lab_batch[:,:,:,0]     
        X_batch = gray2rgb(X_batch)      
        X_batch = X_batch.reshape((-1,IMG_SIZE,IMG_SIZE,3)) 
        
        Y_batch = lab_batch[:,:,:,1:] / 128 
        
        return X_batch, Y_batch




Model.py

In [ ]:
IMG_SIZE =64
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Input

def model_creation(IMG_SIZE, batch_size):

    input_layer= Input(shape=(IMG_SIZE, IMG_SIZE, 3,))

    newmodel = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(input_layer)
    newmodel = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(newmodel)
    newmodel = MaxPool2D(pool_size=(2,2),strides=(2,2))(newmodel)
    newmodel = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = MaxPool2D(pool_size=(2,2),strides=(2,2))(newmodel)
    newmodel = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = MaxPool2D(pool_size=(2,2),strides=(2,2))(newmodel)
    newmodel = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = MaxPool2D(pool_size=(2,2),strides=(2,2))(newmodel)
    newmodel = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(newmodel)
    newmodel = MaxPool2D(pool_size=(2,2),strides=(2,2))(newmodel)

    #newmodel.summary()

    #Encoder
    #encoder_input = Input(shape=(7, 7, 512,)) #Wrong concept


    #Decoder
    decoder_output = Conv2D(256, (3,3),  padding='same')(newmodel)
    decoder_output = BatchNormalization(epsilon=1e-05, momentum=0.9)(decoder_output)
    decoder_output = Activation('relu')(decoder_output)

    decoder_output = Conv2D(128, (3,3),  padding='same')(decoder_output)
    decoder_output = BatchNormalization(epsilon=1e-05, momentum=0.9)(decoder_output)
    decoder_output = Activation('relu')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)

    decoder_output = Conv2D(64, (3,3), padding='same')(decoder_output)
    decoder_output = BatchNormalization(epsilon=1e-05, momentum=0.9)(decoder_output)
    decoder_output = Activation('relu')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)

    decoder_output = Conv2D(32, (3,3), padding='same')(decoder_output)
    decoder_output = BatchNormalization(epsilon=1e-05, momentum=0.9)(decoder_output)
    decoder_output = Activation('relu')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)

    decoder_output = Conv2D(16, (3,3), padding='same')(decoder_output)
    decoder_output = BatchNormalization(epsilon=1e-05, momentum=0.9)(decoder_output)
    decoder_output = Activation('tanh')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)

    decoder_output = Conv2D(2, (3, 3), padding='same')(decoder_output)
    decoder_output = BatchNormalization(epsilon=1e-05, momentum=0.9)(decoder_output)
    decoder_output = Activation('tanh')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)
    model = Model(inputs=input_layer, outputs=decoder_output)

    model.summary()
    return model

##New Default (From Tensorflow Library) Pure Custom DSSIM loss

In [ ]:
def DSSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

compile.py

In [ ]:
model= model_creation(IMG_SIZE, batch_size)
opt = keras.optimizers.Adam(learning_rate=0.003)
murad_optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
#model.compile(optimizer='adam', loss=our_loss_dssim, metrics=[our_eval_ssim]) 
model.compile(optimizer=murad_optimizer,loss=DSSIMLoss, metrics=['mean_squared_error','accuracy']) #New (29/08/2020)
#model.compile(optimizer=opt, loss='mse', metrics=['mean_squared_error']) #MSE d,a:lr=0.007;min loss=0.0074 #MSE b,c:lr=0.003 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)      

train.py

In [ ]:
list_files = []
list_file_pattern = ["Dataset/Train/*.jpg"]
for file_pattern in list_file_pattern:
    sub_list_files = glob.glob(file_pattern)
    list_files = [*list_files, *sub_list_files]


random.seed(10)
random.shuffle(list_files)

N_TEST_SAMPLES = int(0.1 * len(list_files))
N_TRAIN_SAMPLES= len(list_files) - N_TEST_SAMPLES
list_files_train, list_files_val = list_files[:N_TRAIN_SAMPLES], list_files[N_TRAIN_SAMPLES:]

print("Total number of image:", len(list_files))
print("Total number of training:", N_TRAIN_SAMPLES)
print("Total number of testing:", N_TEST_SAMPLES)


params = {
    'dim': (IMG_SIZE, IMG_SIZE, 3),
    #'std': np.array([0.229, 0.224, 0.225]),
    #'mean': np.array([0.485, 0.456, 0.406]),
    'batch_size': batch_size
}

# Generators
training_generator = ImageDataGenerator(list_files_train, **params, shuffle=True)
validation_generator = ImageDataGenerator(list_files_val, **params, shuffle=True)


tensorboard = TensorBoard(log_dir="output/first_run")
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=0.000001)
callback_list=[tensorboard, learning_rate_reduction]


model.fit_generator(
        training_generator,
        steps_per_epoch=min(N_TRAIN_SAMPLES // batch_size, 500),
        initial_epoch=0,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=min(N_TEST_SAMPLES // batch_size, 50),
        callbacks=callback_list,
        verbose=1
    )

#model save
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [ ]:
# load model

from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model.h5')

In [ ]:
# Test images
test_list=[]
for filename in os.listdir('/content/drive/My Drive/Dataset/Test/'):
    test_list.append(img_to_array(load_img('/content/drive/My Drive/Dataset/Test/'+ filename, target_size=(IMG_SIZE, IMG_SIZE))))

test_list = np.array(test_list, dtype=float)
Xtest = rgb2lab(1.0/255*test_list)[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,)) ####why this???
Ytest = rgb2lab(1.0/255*test_list)[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

141/141 [==============================] - 37s 261ms/step
0.005860841465644961


In [ ]:
i = 0

for filename in os.listdir('/content/drive/My Drive/Dataset/Test/'):
    test = img_to_array(load_img('/content/drive/My Drive/Dataset/Test/'+filename, target_size=(IMG_SIZE, IMG_SIZE)) )
    test*= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,IMG_SIZE,IMG_SIZE,3))
    #print(L.shape)
    ab = model.predict(L)
    
    #print(ab.shape)
    ab = ab*128
    cur = np.zeros((IMG_SIZE, IMG_SIZE, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    img = (lab2rgb(cur)*255).astype(np.uint8)
    #dim = (400, 400)
    #resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    imsave("/content/drive/My Drive/Temp2/img"+str(i)+"c.jpg", img)
    i+=1